In [80]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [81]:
url = 'https://www.forbes.ru/biznes'
links = []
driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")

try:
    driver.get(url=url)
    time.sleep(1)
    print(1)
    elem = driver.find_element(By.TAG_NAME, "body")
    print(elem)
    no_of_pagedowns = 50
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_UP)
        time.sleep(0.1)
        elem.send_keys(Keys.END)
        time.sleep(0.2)
        no_of_pagedowns-=1
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')

except Exception as e:
    print(e)
finally:
    driver.close()
    driver.quit()


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_36396\2254764488.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")


1
<selenium.webdriver.remote.webelement.WebElement (session="80ff0a01ff630e7591db2b101257f24f", element="bf24818b-1821-449c-a85e-27e261a7a28c")>


In [82]:
#print(soup.text)
news_links = soup.find_all('a')
links_array = []
for i in range(len(news_links)):
    link = news_links[i].get('href').split('?')[0]
    if link[0] != 'h' and len(link) >= 18 and '@' not in link:
        links_array.append('https://www.forbes.ru' + link)
#print(links_array)
#print(news_links)

In [84]:
#print(soup)
def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('forbes_biznes_data.json', "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def main():
    news_data = {}

    for i in range(len(links_array)):

        link = links_array[i]
        #print(link)
        name = link
        news_data[name] = {}
        soup = get_soup(link)
        #print(soup.text)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find_all(attrs={'itemprop' : 'articleBody'})

        #print(article[0].get_text())

        title = soup.title
        article_text = ''
        try:
            for paragraph in article:
                paragraph = paragraph.get_text()
                article_text+=paragraph
            #print(article_paragraphs)

        except AttributeError:
            continue

        #print(article_text)

        """for paragraph in article_paragraphs:
            #article_text += paragraph.text
            article_text += paragraph
            print(paragraph)
        print(article_text)"""

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        #news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', ' ').replace('\r', ' ')

        try:
            news_data[name]['title'] = title.text
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'


    save_json(news_data)
    print("finished")


pd.read_json(open("forbes_biznes_data.json", encoding='utf-8')).transpose().reset_index(drop=True).to_csv("parced_data/forbes_data.csv")



if __name__ == "__main__":
    main()

finished
